In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
data_dir = '/opt/Malware-Project/BigDataset/FEELScenarios/'

In [29]:
df = pd.read_csv(os.path.join(data_dir, 'Processed', 'Client4', 'Day1', "comb_features_ben.csv"))
df.head()

,num_flows,avg_dur,std_dev_dur,percent_stdev_dur,total_size_of_flows_orig,total_size_of_flows_resp,ratio_of_sizes,percent_of_established_states,inbound_pckts,outbound_pckts,...,ratio_of_differ_subject_in_cert,ratio_of_differ_issuer_in_cert,ratio_of_differ_sandns_in_cert,ratio_of_same_subjects,ratio_of_same_issuer,ratio_is_same_CN_and_SNI,avg_certificate_exponent,is_SNI_in_top_level_domain,ratio_certificate_path_error,ratio_missing_cert_in_cert_path
0,1.0,445.491201,0.000000,0.000000,53020.0,24888.0,0.469408,1.000000,261.0,275.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
1,41.0,63.753415,36.536303,0.414634,36933.0,175896.0,4.762570,1.000000,1602.0,1801.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
2,3.0,15.502029,21.911534,0.333333,646.0,2643.0,4.091331,0.000000,12.0,13.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
3,1.0,10.579209,0.000000,0.000000,1910.0,6732.0,3.524607,1.000000,15.0,22.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
4,3.0,143.061461,133.159468,0.333333,4676.0,236240.0,50.521814,0.666667,92.0,108.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0


In [5]:
df.columns

Index(['num_flows', 'avg_dur', 'std_dev_dur', 'percent_stdev_dur',
       'total_size_of_flows_orig', 'total_size_of_flows_resp',
       'ratio_of_sizes', 'percent_of_established_states', 'inbound_pckts',
       'outbound_pckts', 'periodicity_avg', 'periodicity_stdev', 'ssl_ratio',
       'average_public_key', 'tls_version_ratio', 'avg_cert_length',
       'stdev_cert_length', 'is_valid_certificate_during_capture',
       'amount_diff_certificates', 'num_domains_in_cert', 'cert_ratio',
       'num_certificate_path', 'x509_ssl_ratio', 'SNI_ssl_ratio',
       'self_signed_ratio', 'is_SNIs_in_SNA_dns', 'SNI_equal_DstIP',
       'is_CNs_in_SNA_dns', 'ratio_of_differ_SNI_in_ssl_log',
       'ratio_of_differ_subject_in_ssl_log',
       'ratio_of_differ_issuer_in_ssl_log', 'ratio_of_differ_subject_in_cert',
       'ratio_of_differ_issuer_in_cert', 'ratio_of_differ_sandns_in_cert',
       'ratio_of_same_subjects', 'ratio_of_same_issuer',
       'ratio_is_same_CN_and_SNI', 'avg_certificate_expo

In [17]:
df2 = pd.read_csv(os.path.join(data_dir, 'Raw', 'Malware', 'CTU-Malware-Capture-Botnet-327-2', 'Day2', "comb_features.csv"))
df2.head()

,num_flows,avg_dur,std_dev_dur,percent_stdev_dur,total_size_of_flows_orig,total_size_of_flows_resp,ratio_of_sizes,percent_of_established_states,inbound_pckts,outbound_pckts,...,ratio_of_differ_subject_in_cert,ratio_of_differ_issuer_in_cert,ratio_of_differ_sandns_in_cert,ratio_of_same_subjects,ratio_of_same_issuer,ratio_is_same_CN_and_SNI,avg_certificate_exponent,is_SNI_in_top_level_domain,ratio_certificate_path_error,ratio_missing_cert_in_cert_path
0,1.0,100.450049,0.000000,0.0,1092.0,3754.0,3.437729,1.0,15.0,16.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1.0,1.762660,0.000000,0.0,565.0,3496.0,6.187611,1.0,11.0,11.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,1.559557,0.000000,0.0,501.0,7421.0,14.812375,1.0,10.0,9.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2.0,1.752447,0.000216,0.0,1130.0,6992.0,6.187611,1.0,20.0,20.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2.0,1.299021,0.006157,0.0,1002.0,14842.0,14.812375,1.0,20.0,18.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [21]:
df.ssl_ratio.describe()

count    345.000000
mean       0.318478
std        0.654570
min        0.000000
25%        0.000000
50%        0.000000
75%        0.333333
max        4.000000
Name: ssl_ratio, dtype: float64

In [22]:
df2.ssl_ratio.describe()

count    72.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: ssl_ratio, dtype: float64

# Anomaly detection tests

## Create benign dataset for each day

In [54]:
data = dict()
for j in range(1, 6):
    data["Day"+str(j)] = pd.DataFrame()
    for i in range(1, 10):
        df_temp = pd.read_csv(os.path.join(data_dir, 'Processed', 'Client'+str(i), 'Day'+str(j), "comb_features_ben.csv"))
        data["Day"+str(j)] = pd.concat([data["Day"+str(j)], df_temp], ignore_index=True)
    

In [55]:
for i in range(1, 6):
    print(f'Size of Day{i} dataframe: {len(data["Day"+str(i)])}')

Size of Day1 dataframe: 6807
Size of Day2 dataframe: 5892
Size of Day3 dataframe: 5412
Size of Day4 dataframe: 6085
Size of Day5 dataframe: 6058


In [56]:
# Drop column "ssl_ratio"
for i in range(1, 6):
    data["Day"+str(i)] = data["Day"+str(i)].drop(["ssl_ratio"], axis=1)
    data["Day"+str(i)] = data["Day"+str(i)].drop_duplicates()

In [57]:
# Final sizes
for i in range(1, 6):
    print(f'Size of Day{i} dataframe: {len(data["Day"+str(i)])}')

Size of Day1 dataframe: 6805
Size of Day2 dataframe: 5886
Size of Day3 dataframe: 5409
Size of Day4 dataframe: 6079
Size of Day5 dataframe: 6057


## Create malware dataset (Day1)

In [62]:
mal_data = dict()
mal_folders = ['CTU-Malware-Capture-Botnet-346-1', 'CTU-Malware-Capture-Botnet-327-2', 'CTU-Malware-Capture-Botnet-230-1', 'CTU-Malware-Capture-Botnet-219-2']

for folder in mal_folders:
    mal_data[folder] = pd.DataFrame()
    df_temp = pd.read_csv(os.path.join(data_dir, 'Raw', 'Malware', folder, 'Day1', "comb_features.csv"))
    mal_data[folder] = pd.concat([mal_data[folder], df_temp], ignore_index=True)

In [65]:
for folder in mal_folders:
    print(f'Size of {folder} dataframe: {len(mal_data[folder])}')

Size of CTU-Malware-Capture-Botnet-346-1 dataframe: 48
Size of CTU-Malware-Capture-Botnet-327-2 dataframe: 72
Size of CTU-Malware-Capture-Botnet-230-1 dataframe: 4
Size of CTU-Malware-Capture-Botnet-219-2 dataframe: 21


In [67]:
# Drop column "ssl_ratio"
for folder in mal_folders:
    mal_data[folder] = mal_data[folder].drop(["ssl_ratio"], axis=1)
    mal_data[folder] = mal_data[folder].drop_duplicates()

In [68]:
for folder in mal_folders:
    print(f'Size of {folder} dataframe: {len(mal_data[folder])}')

Size of CTU-Malware-Capture-Botnet-346-1 dataframe: 48
Size of CTU-Malware-Capture-Botnet-327-2 dataframe: 72
Size of CTU-Malware-Capture-Botnet-230-1 dataframe: 4
Size of CTU-Malware-Capture-Botnet-219-2 dataframe: 21


## Anomaly detection

### Isolation Forests

In [151]:
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing

In [284]:
# scaler = preprocessing.StandardScaler().fit(data["Day1"])
scaler = preprocessing.MinMaxScaler().fit(data["Day1"])
X = scaler.transform(data["Day1"])
iso = IsolationForest(n_estimators=50, 
                      contamination=0.02,
                      random_state=1337).fit(X)

In [285]:
X_test = scaler.transform(data["Day2"])
print(f'False positives: {100*sum(iso.predict(X_test) == -1) / len(X_test):.2f}%')

False positives: 1.78%


In [286]:
for folder in mal_folders:
    X_test_mal = scaler.transform(mal_data[folder])
    print(f'Detection {folder}: {100*sum(iso.predict(X_test_mal) == -1) / len(X_test_mal):.2f}%')

Detection CTU-Malware-Capture-Botnet-346-1: 85.42%
Detection CTU-Malware-Capture-Botnet-327-2: 0.00%
Detection CTU-Malware-Capture-Botnet-230-1: 0.00%
Detection CTU-Malware-Capture-Botnet-219-2: 66.67%


### LOF

In [275]:
from sklearn.neighbors import LocalOutlierFactor

In [277]:
lof = LocalOutlierFactor(n_neighbors=10, novelty=True, contamination=0.02)

In [278]:
scaler = preprocessing.StandardScaler().fit(data["Day1"])
# scaler = preprocessing.MinMaxScaler().fit(data["Day1"])
X = scaler.transform(data["Day1"])
lof.fit(X)

LocalOutlierFactor(contamination=0.02, n_neighbors=10, novelty=True)

In [279]:
X_test = scaler.transform(data["Day2"])
print(f'False positives: {100*sum(lof.predict(X_test) == -1) / len(X_test):.2f}%')

False positives: 1.80%


In [280]:
for folder in mal_folders:
    X_test_mal = scaler.transform(mal_data[folder])
    print(f'Detection {folder}: {100*sum(lof.predict(X_test_mal) == -1) / len(X_test_mal):.2f}%')

Detection CTU-Malware-Capture-Botnet-346-1: 4.17%
Detection CTU-Malware-Capture-Botnet-327-2: 0.00%
Detection CTU-Malware-Capture-Botnet-230-1: 25.00%
Detection CTU-Malware-Capture-Botnet-219-2: 28.57%


### One class SVM

In [248]:
from sklearn.svm import OneClassSVM

In [314]:
# scaler = preprocessing.StandardScaler().fit(data["Day2"])
scaler = preprocessing.MinMaxScaler().fit(data["Day1"])
X = scaler.transform(data["Day1"])
svm = OneClassSVM(nu=0.01, kernel="rbf", gamma=0.1).fit(X)

In [323]:
X_test = scaler.transform(data["Day2"])
print(f'False positives: {100*sum(svm.predict(X_test) == -1) / len(X_test):.2f}% ({sum(svm.predict(X_test)==-1)} out of {len(X_test)})')

False positives: 1.60% (94 out of 5886)


In [321]:
for folder in mal_folders:
    X_test_mal = scaler.transform(mal_data[folder])
    print(f'{folder} detected: {100*sum(svm.predict(X_test_mal) == -1) / len(X_test_mal):.2f}% ({sum(svm.predict(X_test_mal)==-1)} out of {len(X_test_mal)})')

CTU-Malware-Capture-Botnet-346-1 detected: 100.00% (48 out of 48)
CTU-Malware-Capture-Botnet-327-2 detected: 100.00% (72 out of 72)
CTU-Malware-Capture-Botnet-230-1 detected: 0.00% (0 out of 4)
CTU-Malware-Capture-Botnet-219-2 detected: 71.43% (15 out of 21)


### Autoencoders (NN)